In [1]:
import torch
import numpy as np
import multiprocessing as mp
import graph_alg3 as ga

import time

In [2]:
def sinkhorn_knopp(matrix, max_iter=1000, tol=1e-6):
    for _ in range(max_iter):
        matrix /= matrix.sum(axis=1, keepdims=True)  # 行ごとに正規化
        matrix /= matrix.sum(axis=0, keepdims=True)  # 列ごとに正規化
        
        if np.allclose(matrix.sum(axis=1), 1, atol=tol) and np.allclose(matrix.sum(axis=0), 1, atol=tol):
            break
    
    return matrix


def generate_permutation_array(N, num_goods):
    P = np.zeros((N, num_goods))
    for i in range(N): 
        P[i] = np.random.permutation(num_goods)
    
    return P


def sample_ranking(N, num_goods):
        """ 
        Samples ranked lists
        Arguments
            N: Number of samples
            prob: Probability of truncation       
        Returns:
            Ranked List of shape [N, Num_agents]
        """
        P = generate_permutation_array(N, num_goods) + 1
        
        return P / num_goods

    
def generate_batch(batch_size, num_goods):
        """
        Samples a batch of data from training
        Arguments
            batch_size: number of samples
            prob: probability of truncation
        Returns
            P: Agent's preferences, 
                P_{ij}: How much Agent-i prefers to be Good-j
        """
        N = batch_size * num_goods
        P = sample_ranking(N, num_goods).reshape(-1, num_goods, num_goods)
        return P

In [ ]:
num_iterations = 100  # 繰り返し回数
batch_size = 1
num_goods = 4

for i in range(num_iterations):
    start_time = time.perf_counter()  # 開始時刻の計測
    
    # ランダムな割り当て行列の生成と処理
    randomassignment = np.random.rand(batch_size, num_goods, num_goods)
    RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
    
    # 好み（preferences）の生成
    preferences = generate_batch(batch_size, num_goods)
    
    # numpy配列から PyTorch の tensor に変換
    P = torch.tensor(RandomAssignment, dtype=torch.float32)
    Preferences = torch.tensor(preferences, dtype=torch.float32)
    
    # 入力データの表示
    print(f"Iteration {i+1}")
    print("P: ", P)
    print("Preferences: ", Preferences)
    
    # グラフアルゴリズムによる処理
    ev = ga.compute_ev(P, Preferences)
    violations, cycles = ev.execute_all_cycles_batch()
    print("Violations:", violations)
    end_time = time.perf_counter()  # 終了時刻の計測
    print(f"Time taken for iteration {i+1}: {end_time - start_time:.6f} sec")
    print("Cycles:", cycles)
    #print("CPU Count:", cpucount)
    
    
    print("-" * 50)